# Tutorial de Keras

Keras es una API de alto nivel para definir y entrenar modelos de redes neuronales totalmente compatible con tensorflow. Viene incluido dentro de tensorflow como el módulo **tf.keras**. 

Keras abstrae el diseño de una red neuronal a partir de las capas que la componen. Toda la mecánica de descenso de gradiente, retropropagación y optimizadores se maneja internamente, por lo que el desarrollador solamente debe preocuparse por definir bien la arquitectura y los hiperparámetros.


## Clasificador de imágenes usando Keras

En este notebook, se construirá una red neuronal feed forward densamente conectada para clasificar datos de dígitos manuscritos.

El dataset usado es el MNIST, un dataset bastante conocido en el mundo de la clasificación de imágenes y es considerado como el hola mundo de las redes neuronales para visión artificial.


In [13]:
# importar el modulo de tensorflow (tensorflow >= 2.0)
import tensorflow as tf

# importar el dataset
mnist = tf.keras.datasets.mnist

# extraer el dataset en tensores
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(f'entrenamiento: {x_train.shape}')
print(f'pruebas: {x_test.shape}')


entrenamiento: (60000, 28, 28)
pruebas: (10000, 28, 28)


## Modelo Secuencial (Sequential)

En Keras, una red neuronal feed-forward se define usando el modelo secuencial. En este modelo, se asume que las capas de la red neuronal vienen una después de otra y se van apilando a medida que se agregan nuevas capas. El modelo secuencial puede ser aplicado en una inmensidad de aplicaciones.

Para construir el modelo necesitamos crear una instancia del objeto **Sequential** y agregar las distintas capas que lo componen.


In [14]:
# crear el modelo y agregar las capas
model = tf.keras.models.Sequential([                
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),        # dense es una capa densamente conectada WX + b
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [15]:
# la salida de la red es una capa lineal, tambien denominados logits.
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.46586984, -0.3597898 ,  0.34318417,  0.25944442,  0.02655937,
        -0.6195074 , -0.53194034,  0.06394144, -0.3848926 ,  0.18875027]],
      dtype=float32)

In [19]:
# para convertir estos logits en una salida softmax aplicamos un operador de tensorflow
tf.nn.softmax(predictions).numpy()
# print(tf.argmax(predictions[0]))

array([[0.1577039 , 0.0690658 , 0.13949567, 0.12829007, 0.10163712,
        0.05326833, 0.05814321, 0.10550843, 0.06735363, 0.11953386]],
      dtype=float32)

### Función de costo

Keras nos ofrece a disposición distintos tipos de funciones de costo incorporados con los cuales podemos trabajar directamente.


In [20]:
# uilizando la entropia cruzada categorica
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [21]:
# podemos hallar el costo de las predicciones
loss_fn(y_train[:1], predictions).numpy()

2.932413

### Compilacion del modelo

Una vez definida la arquitectura y función de costo se procede a *compilar* el modelo, lo cual transforma nuestro objeto en un grafo de cómputo de tensorflow.


In [23]:
model.compile(optimizer='adam',             # algoritmo optimizador (descenso de gradiente)
              loss=loss_fn,                 # funcion de costo
              metrics=['accuracy'])         # metricas para monitorear
model.summary()        

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


### Entrenamiento del modelo

Para entrenar el modelo, una vez definida la arquitectura y los demás parámetros, simplemente se debe ejecutar el método fit sobre nuestro modelo.


In [24]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2973 - accuracy: 0.9140
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1406 - accuracy: 0.9585
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1055 - accuracy: 0.9690
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0877 - accuracy: 0.9728
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0728 - accuracy: 0.9771


### Evaluación del modelo

También existe una función implementada para evaluar el modelo sobre un conjunto de pruebas


In [25]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0675 - accuracy: 0.9787


[0.06751865148544312, 0.9786999821662903]

El modelo ha sido entrenado alcanzando un 98% de precisión